(vis-layers)=
# Layers


## Introduction

In the previous chapters, you've learned much more than just how to make scatterplots, bar charts, and boxplots.
You learned a foundation that you can use to make *any* type of plot with **lets-plot**.

In this chapter, you'll expand on that foundation as you learn about the layered grammar of graphics.
We'll start with a deeper dive into aesthetic mappings, geometric objects, and facets.
Then, you will learn about statistical transformations **lets-plot** makes under the hood when creating a plot.
These transformations are used to calculate new values to plot, such as the heights of bars in a bar plot or medians in a box plot.
You will also learn about position adjustments, which modify how geoms are displayed in your plots.
Finally, we'll briefly introduce coordinate systems.

We will not cover every single function and option for each of these layers, but we will walk you through the most important and commonly used functionality provided by **lets-plot**.

### Prerequisites

You will need to install the **letsplot** package for this chapter, as well as **pandas**.

In your Python session, import the libraries we'll be using:

In [ ]:
import pandas as pd
from lets_plot import *

LetsPlot.setup_html()

## Aesthetic mappings

> "The greatest value of a picture is when it forces us to notice what we never expected to see." --- John Tukey

We're going to use the `mpg` dataset for this section, so let's download it.

In [ ]:
mpg = pd.read_csv(
    "https://vincentarelbundock.github.io/Rdatasets/csv/ggplot2/mpg.csv", index_col=0
)

mpg = mpg.astype(
    {
        "manufacturer": "category",
        "model": "category",
        "displ": "double",
        "year": "int64",
        "cyl": "int64",
        "trans": "category",
        "drv": "category",
        "cty": "double",
        "hwy": "double",
        "fl": "category",
        "class": "category",
    }
)
mpg.head()

Among the variables in `mpg` are:

1.  `displ`: A car's engine size, in liters.
    A numerical variable.

2.  `hwy`: A car's fuel efficiency on the highway, in miles per gallon (mpg).
    A car with a low fuel efficiency consumes more fuel than a car with a high fuel efficiency when they travel the same distance.
    A numerical variable.

3.  `class`: Type of car.
    A categorical variable.

Let's start by visualising the relationship between `displ` and `hwy` for various `class`es of cars.
We can do this with a scatterplot where the numerical variables are mapped to the `x` and `y` aesthetics and the categorical variable is mapped to an aesthetic like `color` or `shape`.

In [ ]:
(ggplot(mpg, aes(x="displ", y="hwy", color="class")) + geom_point())

In [ ]:
(ggplot(mpg, aes(x="displ", y="hwy", shape="class")) + geom_point())

Similarly, we can map `class` to `size` or `alpha` aesthetics as well, which control the shape and the transparency of the points, respectively.

In [ ]:
(ggplot(mpg, aes(x="displ", y="hwy", size="class")) + geom_point())

In [ ]:
(ggplot(mpg, aes(x="displ", y="hwy", alpha="class")) + geom_point())

While we are able to do it, mapping an unordered discrete (categorical) variable (`class`) to an ordered aesthetic variable (`size` or `alpha`) is generally not a good idea because it implies a ranking that does not in fact exist.

Once you map an aesthetic, **lets-plot** takes care of the rest.
It selects a reasonable scale to use with the aesthetic, and it constructs a legend that explains the mapping between levels and values.
For x and y aesthetics, **lets-plot** does not create a legend, but it creates an axis line with tick marks and a label.
The axis line provides the same information as a legend; it explains the mapping between locations and values.

You can also set the visual properties of your geom manually as an argument of your geom function (*outside* of `aes()`) instead of relying on a variable mapping to determine the appearance.
For example, we can make all of the points in our plot blue:


In [ ]:
(ggplot(mpg, aes(x="displ", y="hwy")) + geom_point(color="blue"))

Here, the colour doesn't convey information about a variable, but only changes the appearance of the plot.
You'll need to pick a value that makes sense for that aesthetic:

-   The name of a color as a character string, e.g., `color = "blue"`
-   The size of a point in mm, e.g., `size = 1`
-   The shape of a point as a number, e.g, `shape = 1`.

Try changing the above plot but, instead of specifying colour, try specifying the shape aesthetic. What do you get with shape set to 1, 2, or 3?

So far we have discussed aesthetics that we can map or set in a scatterplot, when using a point geom.

The specific aesthetics you can use for a plot depend on the geom you use to represent the data.
In the next section we dive deeper into geoms.

1.  Create a scatterplot of `hwy` vs. `displ` where the points are pink filled in triangles.

2.  Why does the following code not result in a plot with blue points?

    ```python
    (
        ggplot(mpg) + 
      geom_point(aes(x = "displ", y = "hwy", color = "blue"))
    )
    ```

3.  What does the `stroke` aesthetic do?
    What shapes does it work with?
    (Hint: use `stroke` in the global aesthetic and `shape` in `geom_point`)

4.  Try changing the last plot from above but, instead of specifying colour, try specifying the shape aesthetic. What do you get with shape set to 1, 2, or 3?

## Geometric objects

How are these two plots similar?

In [ ]:
(ggplot(mpg, aes(x="displ", y="hwy")) + geom_point(size=4))

In [ ]:
(ggplot(mpg, aes(x="displ", y="hwy")) + geom_smooth(method="loess", size=2))

Both plots contain the same x variable, the same y variable, and both describe the same data.
But the plots are not identical.
Each plot uses a different geometric object, geom, to represent the data.
The plot on the left uses the point geom, and the plot on the right uses the smooth geom, a smooth line fitted to the data.

To change the geom in your plot, change the geom function that you add to `ggplot()`.

Every geom function in **lets-plot** takes a `mapping` argument, either defined locally in the geom layer or globally in the `ggplot()` layer.
However, not every aesthetic works with every geom.
You could set the shape of a point, but you couldn't set the "shape" of a line.
If you try, **lets-plot** will silently ignore that aesthetic mapping.
On the other hand, you *could* set the linetype of a line.
`geom_smooth()` will draw a different line, with a different linetype, for each unique value of the variable that you map to linetype.

Let's take a look:

In [ ]:
(ggplot(mpg, aes(x="displ", y="hwy", line="drv")) + geom_smooth(method="loess"))

In [ ]:
(ggplot(mpg, aes(x="displ", y="hwy", linetype="drv")) + geom_smooth(method="loess"))

Here, `geom_smooth()` separates the cars into three lines based on their `drv` value, which describes a car's drive train.
One line describes all of the points that have a `4` value, one line describes all of the points that have an `f` value, and one line describes all of the points that have an `r` value.
Here, `4` stands for four-wheel drive, `f` for front-wheel drive, and `r` for rear-wheel drive.

If this is too confusing, we can make it clearer by overlaying the lines on top of the raw data and then coloring everything according to `drv`.

In [ ]:
(
    ggplot(mpg, aes(x="displ", y="hwy", color="drv"))
    + geom_point()
    + geom_smooth(aes(linetype="drv"), method="loess")
)

Notice that this plot contains two geoms in the same graph.

Many geoms, like `geom_smooth()`, use a single geometric object to display multiple rows of data.
For these geoms, you can set the `group` aesthetic to a categorical variable to draw multiple objects.
**lets-plot** will draw a separate object for each unique value of the grouping variable.
In practice, **lets-plot** will automatically group the data for these geoms whenever you map an aesthetic to a discrete variable.
It is convenient to rely on this feature because the `group` aesthetic by itself does not add a legend or distinguishing features to the geoms.

Note that if you place mappings in a geom function, **lets-plot** will treat them as local mappings for the layer.
It will use these mappings to extend or overwrite the global mappings *for that layer only*.
This makes it possible to display different aesthetics in different layers.

In [ ]:
(ggplot(mpg, aes(x="displ", y="hwy")) + geom_point(aes(color="class")) + geom_smooth())

You can use the same idea to specify different data for each layer.
Here, we use red points as well as open circles to highlight two-seater cars.
The local data argument in `geom_point()` overrides the global data argument in `ggplot()` for that layer only.


In [ ]:
(
    ggplot(mpg, aes(x="displ", y="hwy"))
    + geom_point()
    + geom_point(data=mpg.loc[mpg["class"] == "2seater", :], color="red", size=2)
    + geom_point(
        data=mpg.loc[mpg["class"] == "2seater", :], shape=1, size=3, color="red"
    )
)

Geoms are the fundamental building blocks of **lets-plot**.
You can completely transform the look of your plot by changing its geom, and different geoms can reveal different features of your data.

**lets-plot** provides over 40 geoms but these don't cover all possible plots one could make. You can find an overview at the relevant part of the [**lets-plot** documentation](https://lets-plot.org/pages/api.html#geometries).

If you need a geom that is not included, you have three main options:
1. Look for packages that extend **lets-plot** and that do what you need
2. Raise an issue on the [**lets-plot** Github page](https://github.com/JetBrains/lets-plot) requesting it as a new feature—but bear in mind that it might not be a priority for the maintainers, and there's no guarantee that they'll add it, depending on how useful it is for others and how easy it is to implemet.
3. Turn to an imperative plotting package that gives you fine-grained control so you can build your own chart from the ground up—[**matplotlib**](https://matplotlib.org/) is absolutely excellent for this.


### Exercises

1.  What geom would you use to draw a line chart?
    A boxplot?
    A histogram?
    An area chart?

2.  What effect would running the previous example:

    ```python
    (
    ggplot(mpg, aes(x = "displ", y = "hwy", alpha = "class")) +
      geom_point()
    )
    ```
    with the keyword argument `show_legend=False` have on the chart generated by this code?

3.  What does the `se` argument to `geom_smooth()` do?

4.  Recreate the Python code necessary to generate the following graph.


In [ ]:
(
    ggplot(mpg, aes(x="displ", y="hwy", color="drv"))
    + geom_smooth(aes(group="drv"), se=False, method="loess")
    + geom_point()
)